In [1]:
import numpy as np
from intcode_computer import  intcode_computer
from collections import deque

In [2]:
#Check if position is traversable
def good_pos(graph, pos):
    x = pos[0]
    y = pos[1]
    if x < 0 or x >= len(graph):
        return False
    if y < 0 or y >= len(graph[x]):
        return False
    value = graph[x][y]
    return value != 46

#Breadth-First Search
def BFS(graph, start, end=None, get_path=False, get_dist=False, get_junction=False):
    dx = [-1, 1]
    dy = [-1, 1]
    queue = deque([start])
    dist = {start: 0}
    
    junction = []
    
    while len(queue):
        cur_pos = queue.popleft()
        cur_dist = dist[cur_pos]
        if cur_pos == end:
            break
        adj = 0
        for i in range(0, 2):
            nxt_dist = cur_dist + 1
            #move in x
            nxt_pos = (cur_pos[0]+dx[i], cur_pos[1])
            if good_pos(graph, nxt_pos) :
                if nxt_pos not in dist.keys():
                    queue.append(nxt_pos)
                    dist[nxt_pos] = nxt_dist
                adj += 1
            #move in y
            nxt_pos = (cur_pos[0], cur_pos[1]+dy[i])
            if good_pos(graph, nxt_pos):
                if nxt_pos not in dist.keys():
                    queue.append(nxt_pos)
                    dist[nxt_pos] = nxt_dist
                adj += 1
        if adj > 2 and cur_pos not in junction:
            junction.append(cur_pos)
            
    if get_junction:
        return junction
                
    max_dist = 0
    for key in dist.keys():
        if dist[key] > max_dist:
            max_dist = dist[key]
    
    if get_path and end is not None:
        path = [end]
        test_dist = max_dist
        dxy = [[1,0],[-1,0],[0,1],[0,-1]]
        while path[-1] != start:
            for delta in dxy:
                test = (path[-1][0]+delta[0],path[-1][1]+delta[1])
                if test in dist and dist[test] < test_dist:
                    path.append(test)
                    test_dist -= 1
                    break
        return path
    
    if get_dist:
        return dist, max_dist
    
    return max_dist

In [3]:
cpu = intcode_computer(17)

scaf = cpu.run_program(verbose=False)

#convert scaf into nxm array
scaf_arr = []
x = []
for sc in scaf:
    if sc == 10:
        scaf_arr.append(np.array(x))
        x = []
    else:
        x.append(sc)
scaf_arr.pop(-1)
scaf_arr = np.array(scaf_arr)

#find bot to use as starting position
bot = np.where(np.logical_and(scaf_arr != 46, scaf_arr != 35))
bot = (bot[0][0], bot[1][0])

#Use BFS to find junctions
jctn = BFS(scaf_arr, bot, get_junction=True)
for i in range(0, len(jctn)):
    scaf_arr[jctn[i][0],jctn[i][1]] = ord('O')

scaf_str = ''
for i in range(0, len(scaf_arr)):
    for  j in range(0, len(scaf_arr[i])):
        scaf_str += chr(scaf_arr[i,j])
    scaf_str += '\n'
#print(scaf_str)

algn_param = 0
for i in range(0, len(jctn)):
    algn_param += jctn[i][0]*jctn[i][1]
print('Part 1 Solution:', algn_param)

Part 1 Solution: 5680


In [4]:
cpu.reset_memory()
cpu.edit_memory(0, 2)

#By hand
route_str = 'A,B,A,C,A,B,C,A,B,C\n'
A_str = 'R,12,R,4,R,10,R,12\n'
B_str = 'R,6,L,8,R,10\n'
C_str = 'L,8,R,4,R,4,R,6\n'
video_str = 'n\n'

route = []
for char in route_str:
    route.append(ord(char)) 
A = []
for char in A_str:
    A.append(ord(char))
B = []
for char in B_str:
    B.append(ord(char))
C = []
for char in C_str:
    C.append(ord(char))
video = []
for char in video_str:
    video.append(ord(char))
    
inpt = np.hstack((route, A, B, C, video))

dust = cpu.run_program(inpt, verbose=False)

dust_str = ''
for d in dust[:-1]:
    dust_str += chr(d)
#print(dust_str)
print('Part 2 Solution:', dust[-1])

Part 2 Solution: 895965
